## A Simple NN with doc2vec Embedding

## Import Packages and File

In [470]:
import pandas as pd 
import nltk
import numpy as np
#load inthe NTLK stopwords to remove articles, preposition and other words that are not actionable
from nltk.corpus import stopwords
# This allows to create individual objects from a bog of words
from nltk.tokenize import word_tokenize, sent_tokenize
# Lemmatizer helps to reduce words to the base form
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

df=pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/jasmineli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Title Doc2Vec

#### Implementation Doc: https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html

In [471]:
def process_sentence(sentence):
    new_tokens = word_tokenize(sentence)
    new_tokens = [t.lower() for t in new_tokens]
    new_tokens =[t for t in new_tokens if t not in stopwords.words('english')]
    new_tokens = [t for t in new_tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(t) for t in new_tokens]
    return new_tokens

In [472]:
titles = df['title_orig'].tolist()
tokens = [process_sentence(t) for t in titles]
df['title_pre'] = tokens
df.head()

,title,title_orig,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,rating_five_count,...,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,product_url,product_picture,product_id,theme,crawl_month,title_pre
0,2020 Summer Vintage Flamingo Print Pajamas Se...,2020 Summer Vintage Flamingo Print Pajamas Se...,16.00,14,EUR,100,0,3.76,54,26.0,...,4.128521,595097d6a26f6e070cb878d1,0,NaN,https://www.wish.com/c/5e9ae51d43d6a96e303acdb0,https://contestimg.wish.com/api/webimage/5e9ae...,5e9ae51d43d6a96e303acdb0,summer,2020-08,"[summer, vintage, flamingo, print, pajama, set..."
1,SSHOUSE Summer Casual Sleeveless Soirée Party ...,Women's Casual Summer Sleeveless Sexy Mini Dress,8.00,22,EUR,20000,1,3.45,6135,2269.0,...,3.899673,56458aa03a698c35c9050988,0,NaN,https://www.wish.com/c/58940d436a0d3d5da4e95a38,https://contestimg.wish.com/api/webimage/58940...,58940d436a0d3d5da4e95a38,summer,2020-08,"[woman, casual, summer, sleeveless, sexy, mini..."
2,2020 Nouvelle Arrivée Femmes Printemps et Été ...,2020 New Arrival Women Spring and Summer Beach...,8.00,43,EUR,100,0,3.57,14,5.0,...,3.989831,5d464a1ffdf7bc44ee933c65,0,NaN,https://www.wish.com/c/5ea10e2c617580260d55310a,https://contestimg.wish.com/api/webimage/5ea10...,5ea10e2c617580260d55310a,summer,2020-08,"[new, arrival, woman, spring, summer, beach, w..."
3,Hot Summer Cool T-shirt pour les femmes Mode T...,Hot Summer Cool T Shirt for Women Fashion Tops...,8.00,8,EUR,5000,1,4.03,579,295.0,...,4.020435,58cfdefdacb37b556efdff7c,0,NaN,https://www.wish.com/c/5cedf17ad1d44c52c59e4aca,https://contestimg.wish.com/api/webimage/5cedf...,5cedf17ad1d44c52c59e4aca,summer,2020-08,"[hot, summer, cool, shirt, woman, fashion, top..."
4,Femmes Shorts d'été à lacets taille élastique ...,Women Summer Shorts Lace Up Elastic Waistband ...,2.72,3,EUR,100,1,3.10,20,6.0,...,4.001588,5ab3b592c3911a095ad5dadb,0,NaN,https://www.wish.com/c/5ebf5819ebac372b070b0e70,https://contestimg.wish.com/api/webimage/5ebf5...,5ebf5819ebac372b070b0e70,summer,2020-08,"[woman, summer, short, lace, elastic, waistban..."


In [473]:
def tag_data(sentences):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sentences)]
    return tagged_data

In [474]:
data = tag_data(titles)

In [475]:
max_epochs = 100
vec_size = 20
alpha = 0.025

# dm=1 means ‘distributed memory’ (PV-DM) and dm =0 means ‘distributed bag of words’ (PV-DBOW). 
# Distributed Memory model preserves the word order in a document whereas Distributed Bag of words just 
# uses the bag of words approach, which doesn’t preserve any word order.
model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1) 

model.build_vocab(data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [476]:
test_data = word_tokenize("I love Siri".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('71')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['3'])

V1_infer [ 0.02032866 -0.02796199 -0.02156443  0.00342751 -0.00071154 -0.00065618
  0.01527996 -0.01481971  0.02236171 -0.02088824  0.01114816 -0.00908459
  0.01513362  0.00118999  0.016719   -0.00039846  0.02455194 -0.01949126
  0.00690318 -0.00014208]
[('127', 0.6784767508506775), ('764', 0.6265950202941895), ('752', 0.6139605045318604), ('25', 0.5985579490661621), ('789', 0.5886136293411255), ('246', 0.5713376998901367), ('889', 0.5617882013320923), ('1144', 0.5601301789283752), ('436', 0.5425677299499512), ('422', 0.539798378944397)]
[ 0.01457734  0.04639842  0.02174927 -0.02099785  0.01120952 -0.02206228
  0.0288847   0.00915761 -0.01139381 -0.02940559 -0.04013542 -0.00426537
 -0.04469682 -0.04611818 -0.03969955  0.01084536 -0.0325047  -0.03894172
  0.01065583  0.01026331]


/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  
/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  # This is added back by InteractiveShellApp.init_path()


## Preparing Label Y (Low / High Sales)

In [477]:
label = [1 if sales > 200 else 0 for sales in df["units_sold"]]
df['high_sale'] = label

## Metadata

In [478]:
# product color
def main_color(s):
  main_color = {"red":"red", "white":"white", "pink":"pink", "yellow":"yellow", "green":"green", "blue":"blue", "wine":"red", "burgundy":"red", "black":"black", "navy":"navy", "orange":"orange", 
  "rose":"pink", "gray":"gray", "grey":"gray", "purple":"purple", "violet":"purple", "army":"green", "leopard":"orange", "ivory":"white", 
  "brown":"brown", "coffee":"brown", "camel":"beige", "tan":"brown", "nude":"beige", "khaki":"khaki", "apricot":"yellow", "camouflage":"green", "jasper":"red"}  # ordered by importance
  for key, value in main_color.items():
    if key in s:
      return value
  return "others"
product_color = df["product_color"]
product_color = [s.lower() if type(s) is str else 'nan' for s in product_color]
product_color = [main_color(s) for s in product_color]
from matplotlib import colors
product_color = [(-0.1,-0.1,-0.1,-0.1) if s == "others" else colors.to_rgba(s) for s in product_color]

df['product_color_rgb'] = [np.array(t) for t in product_color]

# log prices
df['log_price'] = [np.log(p) for p in df["price"]]
df['log_retail_price'] = [np.log(p) for p in df["retail_price"]]

# log merchant rating count
df['log_merchant_rating_count'] = np.log(df['merchant_rating_count'])

# urgent text
df['urgent'] = [1 if s == "Quantité limitée !" else 0 for s in df["urgency_text"]]

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Feature Engineering 

In [479]:
data = df[["log_price", "log_retail_price", "uses_ad_boosts", "badges_count", "badge_local_product", 
           "badge_product_quality", "badge_fast_shipping", "urgent", 
           "high_sale"]]
# generating doc2vec encoding for preprossed titles
df['doc2vec'] = [model.infer_vector(title) for title in df['title_pre']]
doc2vec = np.stack(df['doc2vec'].values, axis=0)
data2 = pd.DataFrame()
for i in range(20):
    data["doc2vec"+str(i)] = doc2vec[:,i]
    # data for only using titile text as input
    data2["doc2vec"+str(i)] = doc2vec[:,i]
rgb = df["product_color_rgb"]
rgb = np.stack(rgb.values, axis=0)
for i in range(4):
    data["product_color_rgb"+str(i)] = rgb[:,i]

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [480]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


### 1. Building NN Model with Doc2vec

In [481]:
# X = data.loc[:, ~data.columns.isin(['high_sale'])]
X = data2
y = data['high_sale']

In [482]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.14286, random_state=42)

In [483]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of dev examples = " + str(X_dev.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(y_dev.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 1179
number of dev examples = 197
number of test examples = 197
X_train shape: (1179, 20)
Y_train shape: (1179,)
X_dev shape: (197, 20)
Y_dev shape: (197,)
X_test shape: (197, 20)
Y_test shape: (197,)


In [484]:
# from sklearn.linear_model import LogisticRegression

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# score = classifier.score(X_test, y_test)

# print("Accuracy:", score)

In [485]:
# Defining hold out data for evaluation 
evals_X = X[-100:]
evals_y = y[-100:]
print(evals_y.shape)
print(evals_X.shape)
X_train = tf.expand_dims(X_train, axis=-1)
input_shape = X_train.shape[1:]

(100,)
(100, 20)


In [486]:
# define model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [488]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [492]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    verbose=2,
    validation_data=(evals_X, evals_y),
    batch_size=32,
    callbacks=[es]
)
model.summary()

Epoch 1/1000
37/37 - 0s - loss: 0.6046 - accuracy: 0.6675 - val_loss: 0.6071 - val_accuracy: 0.6900 - 93ms/epoch - 3ms/step
Epoch 2/1000
37/37 - 0s - loss: 0.6114 - accuracy: 0.6590 - val_loss: 0.6017 - val_accuracy: 0.7000 - 65ms/epoch - 2ms/step
Epoch 3/1000
37/37 - 0s - loss: 0.6064 - accuracy: 0.6573 - val_loss: 0.6026 - val_accuracy: 0.7100 - 60ms/epoch - 2ms/step
Epoch 4/1000
37/37 - 0s - loss: 0.6062 - accuracy: 0.6641 - val_loss: 0.6028 - val_accuracy: 0.6900 - 61ms/epoch - 2ms/step
Epoch 5/1000
37/37 - 0s - loss: 0.6164 - accuracy: 0.6633 - val_loss: 0.6021 - val_accuracy: 0.6900 - 64ms/epoch - 2ms/step
Epoch 6/1000
37/37 - 0s - loss: 0.6105 - accuracy: 0.6675 - val_loss: 0.5989 - val_accuracy: 0.6900 - 61ms/epoch - 2ms/step
Epoch 7/1000
37/37 - 0s - loss: 0.6077 - accuracy: 0.6624 - val_loss: 0.5970 - val_accuracy: 0.6900 - 63ms/epoch - 2ms/step
Epoch 8/1000
37/37 - 0s - loss: 0.6138 - accuracy: 0.6582 - val_loss: 0.6038 - val_accuracy: 0.7100 - 63ms/epoch - 2ms/step
Epoch 9/

Epoch 67/1000
37/37 - 0s - loss: 0.6006 - accuracy: 0.6735 - val_loss: 0.5879 - val_accuracy: 0.7000 - 63ms/epoch - 2ms/step
Epoch 68/1000
37/37 - 0s - loss: 0.5974 - accuracy: 0.6684 - val_loss: 0.5968 - val_accuracy: 0.7100 - 63ms/epoch - 2ms/step
Epoch 69/1000
37/37 - 0s - loss: 0.5979 - accuracy: 0.6760 - val_loss: 0.6029 - val_accuracy: 0.7000 - 61ms/epoch - 2ms/step
Epoch 70/1000
37/37 - 0s - loss: 0.5963 - accuracy: 0.6760 - val_loss: 0.5794 - val_accuracy: 0.7100 - 63ms/epoch - 2ms/step
Epoch 71/1000
37/37 - 0s - loss: 0.5987 - accuracy: 0.6785 - val_loss: 0.5778 - val_accuracy: 0.7200 - 64ms/epoch - 2ms/step
Epoch 72/1000
37/37 - 0s - loss: 0.6074 - accuracy: 0.6726 - val_loss: 0.6026 - val_accuracy: 0.6900 - 61ms/epoch - 2ms/step
Epoch 73/1000
37/37 - 0s - loss: 0.6097 - accuracy: 0.6667 - val_loss: 0.5859 - val_accuracy: 0.7000 - 59ms/epoch - 2ms/step
Epoch 74/1000
37/37 - 0s - loss: 0.5986 - accuracy: 0.6802 - val_loss: 0.5826 - val_accuracy: 0.7100 - 63ms/epoch - 2ms/step


Epoch 133/1000
37/37 - 0s - loss: 0.5758 - accuracy: 0.6997 - val_loss: 0.5746 - val_accuracy: 0.7100 - 64ms/epoch - 2ms/step
Epoch 134/1000
37/37 - 0s - loss: 0.6006 - accuracy: 0.6667 - val_loss: 0.5643 - val_accuracy: 0.7300 - 63ms/epoch - 2ms/step
Epoch 135/1000
37/37 - 0s - loss: 0.5940 - accuracy: 0.6913 - val_loss: 0.5665 - val_accuracy: 0.7400 - 62ms/epoch - 2ms/step
Epoch 136/1000
37/37 - 0s - loss: 0.5940 - accuracy: 0.6819 - val_loss: 0.5813 - val_accuracy: 0.7300 - 63ms/epoch - 2ms/step
Epoch 137/1000
37/37 - 0s - loss: 0.5858 - accuracy: 0.6904 - val_loss: 0.5835 - val_accuracy: 0.7100 - 60ms/epoch - 2ms/step
Epoch 138/1000
37/37 - 0s - loss: 0.5925 - accuracy: 0.6785 - val_loss: 0.5673 - val_accuracy: 0.7200 - 59ms/epoch - 2ms/step
Epoch 139/1000
37/37 - 0s - loss: 0.5907 - accuracy: 0.6870 - val_loss: 0.5830 - val_accuracy: 0.7400 - 59ms/epoch - 2ms/step
Epoch 140/1000
37/37 - 0s - loss: 0.5942 - accuracy: 0.6650 - val_loss: 0.5824 - val_accuracy: 0.7000 - 62ms/epoch - 2

37/37 - 0s - loss: 0.5772 - accuracy: 0.7048 - val_loss: 0.5642 - val_accuracy: 0.7300 - 58ms/epoch - 2ms/step
Epoch 199/1000
37/37 - 0s - loss: 0.5791 - accuracy: 0.6947 - val_loss: 0.5708 - val_accuracy: 0.7300 - 59ms/epoch - 2ms/step
Epoch 200/1000
37/37 - 0s - loss: 0.5712 - accuracy: 0.6904 - val_loss: 0.5582 - val_accuracy: 0.7600 - 62ms/epoch - 2ms/step
Epoch 201/1000
37/37 - 0s - loss: 0.5889 - accuracy: 0.6828 - val_loss: 0.5541 - val_accuracy: 0.7400 - 62ms/epoch - 2ms/step
Epoch 202/1000
37/37 - 0s - loss: 0.5789 - accuracy: 0.6930 - val_loss: 0.5768 - val_accuracy: 0.7100 - 63ms/epoch - 2ms/step
Epoch 203/1000
37/37 - 0s - loss: 0.5910 - accuracy: 0.6828 - val_loss: 0.5708 - val_accuracy: 0.7100 - 61ms/epoch - 2ms/step
Epoch 204/1000
37/37 - 0s - loss: 0.5671 - accuracy: 0.7065 - val_loss: 0.5602 - val_accuracy: 0.7400 - 61ms/epoch - 2ms/step
Epoch 205/1000
37/37 - 0s - loss: 0.5766 - accuracy: 0.7031 - val_loss: 0.5597 - val_accuracy: 0.7200 - 60ms/epoch - 2ms/step
Epoch 2

Epoch 264/1000
37/37 - 0s - loss: 0.5757 - accuracy: 0.6980 - val_loss: 0.5414 - val_accuracy: 0.7400 - 58ms/epoch - 2ms/step
Epoch 265/1000
37/37 - 0s - loss: 0.5693 - accuracy: 0.7048 - val_loss: 0.5483 - val_accuracy: 0.7400 - 57ms/epoch - 2ms/step
Epoch 266/1000
37/37 - 0s - loss: 0.5713 - accuracy: 0.7091 - val_loss: 0.5527 - val_accuracy: 0.7300 - 59ms/epoch - 2ms/step
Epoch 267/1000
37/37 - 0s - loss: 0.5660 - accuracy: 0.7074 - val_loss: 0.5495 - val_accuracy: 0.7400 - 61ms/epoch - 2ms/step
Epoch 268/1000
37/37 - 0s - loss: 0.5677 - accuracy: 0.7184 - val_loss: 0.5390 - val_accuracy: 0.7500 - 61ms/epoch - 2ms/step
Epoch 269/1000
37/37 - 0s - loss: 0.5647 - accuracy: 0.7074 - val_loss: 0.5382 - val_accuracy: 0.7600 - 62ms/epoch - 2ms/step
Epoch 270/1000
37/37 - 0s - loss: 0.5651 - accuracy: 0.6964 - val_loss: 0.5392 - val_accuracy: 0.7400 - 64ms/epoch - 2ms/step
Epoch 271/1000
37/37 - 0s - loss: 0.5650 - accuracy: 0.7091 - val_loss: 0.5458 - val_accuracy: 0.7400 - 60ms/epoch - 2

...

37/37 - 0s - loss: 0.5281 - accuracy: 0.7226 - val_loss: 0.5068 - val_accuracy: 0.7600 - 61ms/epoch - 2ms/step
Epoch 461/1000
37/37 - 0s - loss: 0.5344 - accuracy: 0.7328 - val_loss: 0.5165 - val_accuracy: 0.7700 - 60ms/epoch - 2ms/step
Epoch 462/1000
37/37 - 0s - loss: 0.5184 - accuracy: 0.7405 - val_loss: 0.5015 - val_accuracy: 0.7700 - 61ms/epoch - 2ms/step
Epoch 463/1000
37/37 - 0s - loss: 0.5374 - accuracy: 0.7167 - val_loss: 0.5081 - val_accuracy: 0.7700 - 59ms/epoch - 2ms/step
Epoch 464/1000
37/37 - 0s - loss: 0.5415 - accuracy: 0.7405 - val_loss: 0.5145 - val_accuracy: 0.7700 - 61ms/epoch - 2ms/step
Epoch 465/1000
37/37 - 0s - loss: 0.5340 - accuracy: 0.7345 - val_loss: 0.5097 - val_accuracy: 0.7700 - 60ms/epoch - 2ms/step
Epoch 466/1000
37/37 - 0s - loss: 0.5339 - accuracy: 0.7311 - val_loss: 0.5083 - val_accuracy: 0.7600 - 68ms/epoch - 2ms/step
Epoch 467/1000
37/37 - 0s - loss: 0.5345 - accuracy: 0.7447 - val_loss: 0.5172 - val_accuracy: 0.7500 - 64ms/epoch - 2ms/step
Epoch 4

Epoch 526/1000
37/37 - 0s - loss: 0.5311 - accuracy: 0.7218 - val_loss: 0.4842 - val_accuracy: 0.7600 - 57ms/epoch - 2ms/step
Epoch 527/1000
37/37 - 0s - loss: 0.5239 - accuracy: 0.7379 - val_loss: 0.4924 - val_accuracy: 0.7700 - 58ms/epoch - 2ms/step
Epoch 528/1000
37/37 - 0s - loss: 0.5461 - accuracy: 0.7176 - val_loss: 0.4808 - val_accuracy: 0.7500 - 60ms/epoch - 2ms/step
Epoch 529/1000
37/37 - 0s - loss: 0.5235 - accuracy: 0.7422 - val_loss: 0.4876 - val_accuracy: 0.7700 - 58ms/epoch - 2ms/step
Epoch 530/1000
37/37 - 0s - loss: 0.5186 - accuracy: 0.7566 - val_loss: 0.4924 - val_accuracy: 0.7700 - 62ms/epoch - 2ms/step
Epoch 531/1000
37/37 - 0s - loss: 0.5166 - accuracy: 0.7439 - val_loss: 0.5136 - val_accuracy: 0.7400 - 62ms/epoch - 2ms/step
Epoch 532/1000
37/37 - 0s - loss: 0.5133 - accuracy: 0.7472 - val_loss: 0.4927 - val_accuracy: 0.7500 - 56ms/epoch - 2ms/step
Epoch 533/1000
37/37 - 0s - loss: 0.5204 - accuracy: 0.7252 - val_loss: 0.4975 - val_accuracy: 0.7700 - 59ms/epoch - 2

37/37 - 0s - loss: 0.5081 - accuracy: 0.7583 - val_loss: 0.4887 - val_accuracy: 0.7700 - 62ms/epoch - 2ms/step
Epoch 592/1000
37/37 - 0s - loss: 0.5100 - accuracy: 0.7506 - val_loss: 0.4777 - val_accuracy: 0.7600 - 60ms/epoch - 2ms/step
Epoch 593/1000
37/37 - 0s - loss: 0.5272 - accuracy: 0.7405 - val_loss: 0.4837 - val_accuracy: 0.7500 - 61ms/epoch - 2ms/step
Epoch 594/1000
37/37 - 0s - loss: 0.5102 - accuracy: 0.7557 - val_loss: 0.4923 - val_accuracy: 0.7700 - 63ms/epoch - 2ms/step
Epoch 595/1000
37/37 - 0s - loss: 0.5094 - accuracy: 0.7515 - val_loss: 0.4743 - val_accuracy: 0.7800 - 60ms/epoch - 2ms/step
Epoch 596/1000
37/37 - 0s - loss: 0.5130 - accuracy: 0.7405 - val_loss: 0.5022 - val_accuracy: 0.7800 - 61ms/epoch - 2ms/step
Epoch 597/1000
37/37 - 0s - loss: 0.5219 - accuracy: 0.7506 - val_loss: 0.4935 - val_accuracy: 0.7500 - 57ms/epoch - 2ms/step
Epoch 598/1000
37/37 - 0s - loss: 0.5381 - accuracy: 0.7226 - val_loss: 0.4914 - val_accuracy: 0.7800 - 59ms/epoch - 2ms/step
Epoch 5

Epoch 657/1000
37/37 - 0s - loss: 0.5106 - accuracy: 0.7396 - val_loss: 0.4711 - val_accuracy: 0.7800 - 58ms/epoch - 2ms/step
Epoch 658/1000
37/37 - 0s - loss: 0.5075 - accuracy: 0.7472 - val_loss: 0.4740 - val_accuracy: 0.7900 - 59ms/epoch - 2ms/step
Epoch 659/1000
37/37 - 0s - loss: 0.5104 - accuracy: 0.7498 - val_loss: 0.4885 - val_accuracy: 0.7700 - 59ms/epoch - 2ms/step
Epoch 660/1000
37/37 - 0s - loss: 0.5078 - accuracy: 0.7718 - val_loss: 0.4651 - val_accuracy: 0.7700 - 62ms/epoch - 2ms/step
Epoch 661/1000
37/37 - 0s - loss: 0.4966 - accuracy: 0.7413 - val_loss: 0.4602 - val_accuracy: 0.7900 - 57ms/epoch - 2ms/step
Epoch 662/1000
37/37 - 0s - loss: 0.5048 - accuracy: 0.7540 - val_loss: 0.4656 - val_accuracy: 0.7800 - 58ms/epoch - 2ms/step
Epoch 663/1000
37/37 - 0s - loss: 0.5006 - accuracy: 0.7566 - val_loss: 0.4550 - val_accuracy: 0.8200 - 60ms/epoch - 2ms/step
Epoch 664/1000
37/37 - 0s - loss: 0.4966 - accuracy: 0.7566 - val_loss: 0.4699 - val_accuracy: 0.8000 - 60ms/epoch - 2

37/37 - 0s - loss: 0.4853 - accuracy: 0.7701 - val_loss: 0.4582 - val_accuracy: 0.8000 - 60ms/epoch - 2ms/step
Epoch 723/1000
37/37 - 0s - loss: 0.4877 - accuracy: 0.7651 - val_loss: 0.4752 - val_accuracy: 0.7900 - 60ms/epoch - 2ms/step
Epoch 724/1000
37/37 - 0s - loss: 0.4852 - accuracy: 0.7642 - val_loss: 0.4461 - val_accuracy: 0.7900 - 59ms/epoch - 2ms/step
Epoch 725/1000
37/37 - 0s - loss: 0.4802 - accuracy: 0.7727 - val_loss: 0.4640 - val_accuracy: 0.7700 - 58ms/epoch - 2ms/step
Epoch 726/1000
37/37 - 0s - loss: 0.5011 - accuracy: 0.7608 - val_loss: 0.4568 - val_accuracy: 0.7800 - 57ms/epoch - 2ms/step
Epoch 727/1000
37/37 - 0s - loss: 0.4933 - accuracy: 0.7608 - val_loss: 0.4580 - val_accuracy: 0.7900 - 59ms/epoch - 2ms/step
Epoch 728/1000
37/37 - 0s - loss: 0.5010 - accuracy: 0.7371 - val_loss: 0.4737 - val_accuracy: 0.7900 - 60ms/epoch - 2ms/step
Epoch 729/1000
37/37 - 0s - loss: 0.4904 - accuracy: 0.7659 - val_loss: 0.4512 - val_accuracy: 0.7900 - 60ms/epoch - 2ms/step
Epoch 7

### LogisticRegression